In [84]:
import model.load_data as ld
import model.scoring_metrics as sm
import importlib
importlib.reload(sm)
importlib.reload(ld)

import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.tree import DecisionTreeClassifier
from sklearn import tree

import sys
np.set_printoptions(threshold=sys.maxsize)
pd.set_option('display.max_columns', None)

In [85]:
datasets = """Bed002 Bed003 Bed004 Bed005 Bed006 Bed008 Bed009 Bed010 Bed011 Bed012 Bed013 Bed014 Bed015 Bed016 Bed017 Bmr001 Bmr002 Bmr005 Bmr007 Bmr009 Bmr010 Bmr011 Bmr012 Bmr013 Bmr014 Bmr018 Bmr019 Bmr021 Bmr022 Bmr024 Bmr025 Bmr026 Bmr027 Bmr029 Bns001 Bns002""".split(" ")
results_merged_path = "../results_merged_fixedf0/"

X_train, y_train, X_test, y_test = ld.train_test_split(datasets,results_merged_path,0.3)

all_features = ['pause', 'speakerChange', 'similarity', 'f0_diff', 'f0_baseline_diff']
features_selected = ['pause', 'speakerChange', 'similarity', 'f0_diff', 'f0_baseline_diff']

X_train = X_train[features_selected]
X_test = X_test[features_selected]

def count_duplicate(df1,df2,feature):
    duplicate_count = 0
    for index, row in df2.iterrows():
        if df1.iloc[index][feature]==df2.iloc[index][feature]:
            duplicate_count+=1
    return duplicate_count

for feature in features_selected:
    print(count_duplicate(X_train,X_test,feature))


25
8609
1
4450
2024


In [86]:
def print_eval(y_pred,y_true):
    k = int(max(1,np.floor((len(y_true)+1)/(2*(sum(y_true)+1)))))
    print('k =',k)

    int_y_pred = (np.array(y_pred))
    int_y_true = (np.array(y_true))

    print('- windiff:',sm.get_windiff(int_y_true,int_y_pred,k))
    print('- pk:',sm.get_pk(int_y_true,int_y_pred,k))
    print('- kkappa:',sm.get_k_kappa(int_y_true,int_y_pred,k))

In [87]:
def DecTree(X_train, X_test, y_train):
    
    clf = DecisionTreeClassifier()
    clf.fit(X_train, y_train)
    y_predicted = clf.predict(X_test)

    # # get importance
    # importance = clf.feature_importances_
    # # summarize feature importance
    # for i,v in enumerate(importance):
    #     print('Feature:',features_selected[i],'->',v)
    # # plot feature importance
    # pyplot.bar(features_selected, importance)
    # pyplot.title('Feature Importance')
    # pyplot.show()

    # # plot tree
    # plt.figure(figsize=(20,10))  # set plot size (denoted in inches)
    # tree.plot_tree(clf, max_depth=4, fontsize=10,feature_names=features_selected)
    # plt.show()
    
    return y_predicted

In [88]:
DT_y_predicted = DecTree(X_train,X_test,y_train)
print_eval(DT_y_predicted,y_test)

k = 54
- windiff: 0.5334220555320307
- pk: 0.49464642187405483
- kkappa: -0.038519589084739646
